In [1]:
import tensorflow as tf
import tensorflow.python.keras as keras
import os

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

train_dir = 'data/train'
validation_dir = 'data/validation'
test_dir = 'data/test'

dirs = [train_dir, validation_dir, test_dir]

for dir in dirs:
    for subdir, dirs, files in os.walk(dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                img = Image.open(file_path)  # try to open the image
                img.verify()  # verify that it is, in fact an image
            except (IOError, SyntaxError) as e:
                print('Bad file:', file_path)  # print out the names of corrupt files
                os.remove(file_path)  # uncomment this line to delete these files

# Set up parameters for datasets
target_size = (150, 150)  # Resize images to 150x150
batch_size = 32

# Create an ImageDataGenerator for the training set
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,  # All images will be resized to target_size
    batch_size=batch_size,
    class_mode='binary')  # Since you have two classes, use binary

# Create an ImageDataGenerator for the validation set
validation_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,  # All images will be resized to target_size
    batch_size=batch_size,
    class_mode='binary')  # Since you have two classes, use binary

# Create an ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,  # All images will be resized to target_size
    batch_size=batch_size,
    class_mode='binary')  # Since you have two classes, use binary

Bad file: data/train/.DS_Store
Bad file: data/validation/.DS_Store
Bad file: data/test/.DS_Store
Found 310 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 11 images belonging to 2 classes.


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)), # Input shape matches the target_size and 3 color channels
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # For binary classification, we can use a sigmoid activation function in the output layer
])

# You can see the details of the model using model.summary()
model.summary()

# Compiling the model
model.compile(loss='binary_crossentropy',  # Binary crossentropy is a common choice for binary classification
              optimizer='adam',
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [4]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # // operator in python performs integer division
    epochs=30,  # You might need to adjust the number of epochs based on your specific use case
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/30


2023-05-26 12:42:16.517572: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
/Users/wachowiak/My Drive (alexander.c.wachowiak@gmail.com)/Documents/Projects/BiteScan/myenv/lib/python3.11/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


9/9 [==============================] - 3s 292ms/step - loss: 0.7228 - accuracy: 0.4784


2023-05-26 12:42:19.931592: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 2/30
9/9 [==============================] - 3s 380ms/step - loss: 0.6938 - accuracy: 0.5108
Epoch 3/30
9/9 [==============================] - 3s 353ms/step - loss: 0.6856 - accuracy: 0.5468
Epoch 4/30
9/9 [==============================] - 3s 337ms/step - loss: 0.6705 - accuracy: 0.5863
Epoch 5/30
9/9 [==============================] - 4s 417ms/step - loss: 0.6572 - accuracy: 0.6439
Epoch 6/30
9/9 [==============================] - 4s 433ms/step - loss: 0.6364 - accuracy: 0.6389
Epoch 7/30
9/9 [==============================] - 4s 419ms/step - loss: 0.6040 - accuracy: 0.6736
Epoch 8/30
9/9 [==============================] - 3s 363ms/step - loss: 0.6176 - accuracy: 0.6458
Epoch 9/30
9/9 [==============================] - 3s 377ms/step - loss: 0.5864 - accuracy: 0.6655
Epoch 10/30
9/9 [==============================] - 3s 367ms/step - loss: 0.5567 - accuracy: 0.7194
Epoch 11/30
9/9 [==============================] - 3s 384ms/step - loss: 0.5565 - accuracy: 0.6910
Epoch 12/30
9/9 [=

In [5]:
if test_generator.samples // batch_size > 0:
    steps = test_generator.samples // batch_size
else:
    steps = 1

test_loss, test_acc = model.evaluate(test_generator, steps=steps)
print('Test accuracy:', test_acc)

1/1 [==============================] - 0s 152ms/step - loss: 0.1838 - accuracy: 0.9091
Test accuracy: 0.9090909361839294


2023-05-26 12:44:05.532574: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [11]:
import tensorflow as tf
from tensorflow import keras
import os

models_dir = './models'
os.makedirs(os.path.join(models_dir, 'full'), exist_ok=True)  # Create the full models directory if it doesn't exist
os.makedirs(os.path.join(models_dir, 'lite'), exist_ok=True)  # Create the lite models directory if it doesn't exist

# Save the trained model in the models directory
model_name = 'model_tick_mosquito_classifier'

full_model_name = 'full_' + model_name
lite_model_name = 'lite_' + model_name

model.save(os.path.join(models_dir, 'full', full_model_name + '.h5'))

model.save(os.path.join(models_dir, 'lite', lite_model_name + '.h5'))

# Convert the Keras model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open(os.path.join(models_dir, 'lite', lite_model_name + '.tflite'), 'wb') as f:
    f.write(tflite_model)

print('Full model saved to:', os.path.join(models_dir, 'full', full_model_name + '.h5'))
print('Lite model saved to:', os.path.join(models_dir, 'lite', lite_model_name + '.tflite'))

/Users/wachowiak/My Drive (alexander.c.wachowiak@gmail.com)/Documents/Projects/BiteScan/myenv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /var/folders/0f/whfmwdtx51n29d36d1wck4j00000gn/T/tmpp4s44kcg/assets


INFO:tensorflow:Assets written to: /var/folders/0f/whfmwdtx51n29d36d1wck4j00000gn/T/tmpp4s44kcg/assets


Full model saved to: ./models/full/full_model_tick_mosquito_classifier.h5
Lite model saved to: ./models/lite/lite_model_tick_mosquito_classifier.tflite


2023-05-26 12:51:15.154343: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-26 12:51:15.154353: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-26 12:51:15.154463: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/0f/whfmwdtx51n29d36d1wck4j00000gn/T/tmpp4s44kcg
2023-05-26 12:51:15.155377: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-05-26 12:51:15.155382: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/0f/whfmwdtx51n29d36d1wck4j00000gn/T/tmpp4s44kcg
2023-05-26 12:51:15.158469: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-05-26 12:51:15.232299: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/0f/whfmwdtx51n29d36d1wck4j00000gn/T/tmpp4s44kcg
2023-05-